In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import torch

In [3]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])

In [4]:
points.shape

torch.Size([3, 2])

In [5]:
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

### stride의 정의 : 각 차원이 1증가할 때 storage 배열상에서 건너 뛰어야하는 원소의 개수의 튜플
    
- points의 경우, stride가 (2,1)임.
- 이는 첫번째 차원이 1증가할때 storage에서 2개의 원소를 건너뛰어야하고, 두번째 차원의 경우 1개의 원소를 건너뜀을 의미

In [6]:
points.stride() 

(2, 1)

In [7]:
points.storage().tolist().index(points[1, 0])

2

In [8]:
!ls

01_contigous_tensors.ipynb noncontiguous_tensor.png
contiguous_tensor.png


### contiguous의 정의
>Contiguous(인접한, 근접한)는 단어의 뜻처럼 Tensor의 각 값들이 메모리에도 순차적으로 저장되어 있는지 여부

| ![](contiguous_tensor.png) |
|:--:|
| <b>contiguous tensor: 요소들이 메모리에 연속적으로 저장되어있음</b>|

| ![](noncontiguous_tensor.png) |
|:--:|
| <b>noncontiguous tensor: 이런식으로 요소들이 메모리에 연속적으로 저장되어 있지 않으면 contiguous하지 않음
</b>|

- view 함수는 contigous한 텐서에 대해서만 사용가능
- 파이토치에서 contigous관련 warning을 줌
- contiguous 텐서는 값 순회 시 띄엄띄엄 참조하지 않기 때문에 데이터 지역성 관점에서 CPU 메모리 접근효율 좋음

In [9]:
points.is_contiguous()

True

In [10]:
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [11]:
points.stride()

(2, 1)

`points` 텐서는 메모리에 연속적으로 저장되어있을 텐데, (통상적으로 contiguous 텐서의 stride는 오른쪽 차원에서 시작해서 증가함)

In [12]:
points_t = points.T

여기서 transpose를 해주면 

In [13]:
points_t.stride()

(1, 2)

stride 값이 뒤바뀌면서 contigous해지지 않게 된다!

In [14]:
points_t.is_contiguous()

False

torch의 contiguous() 함수를 통해 contiguous 텐서로 변환할 수 있다

In [15]:
points_t_cont = points_t.contiguous()

stride가 contiguous하게 변경된 것을 볼 수 있고

In [16]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [17]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

contiguous()함수를 사용하기 전과 비교하여 storage가 재편성 된 것을 볼 수있고 이에 따라 stride도 변경 되었다

In [18]:
points_t_cont.stride() # (1,3) -> (3,1)로 변경

(3, 1)

3차원 텐서에서도 contigousity를 체크해보면

In [19]:
any_t = torch.arange(18).view(2,3,3)
any_t.stride()

(9, 3, 1)

In [20]:
any_t.is_contiguous()

True

2차원때와 마찬가지로 `any_t` 텐서는 stride가 오른쪽 차원에서 시작해서 증가하기 때문에 contigous하고

In [21]:
any_t_perm = any_t.permute(0,2,1)
any_t_perm.stride()

(9, 1, 3)

.permute() 함수로 차원을 swap해준뒤 stride가 contiguousity 규칙에 위반되어 non-contiguous한 것을 확인

In [22]:
any_t_perm.is_contiguous()

False

noncontigous한 `any_t_perm` 텐서에 view 함수를 적용해보면

In [23]:
any_t_perm.view(2, -1) # (2, 3*3)으로 shape를 변경 시도

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

contigous하지 않다고 오류가 난다

In [24]:
any_t.view(2, -1).shape # (2, 3*3)으로 shape 변경 시도

torch.Size([2, 9])

contigous 텐서인 `any_t`는 view가 잘 적용되는 것을 볼 수 있다

torch의 contiguous() 함수를 통해 contiguous 텐서로 변환해주면 view를 사용할 때 발생하는 오류를 해결할 수 있다

In [25]:
any_t_perm_contigous = any_t_perm.contiguous()
any_t_perm_contigous.is_contiguous()

True

In [26]:
any_t_perm_contigous.view(2, -1).shape # (2, 3*3)으로 shape 변경 시도

torch.Size([2, 9])

오류없이 성공적으로 shape를 변경 가능한 것을 볼수 있다

In [27]:
any_t_perm_contigous.stride()

(9, 3, 1)

stride를 확인해보면 contigous하게 바뀐 것을 확인 할 수 있다

In [28]:
any_t_perm.storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [29]:
any_t_perm_contigous.storage()

 0
 3
 6
 1
 4
 7
 2
 5
 8
 9
 12
 15
 10
 13
 16
 11
 14
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

또한, storage도 재배치 된 것을 확인할 수 있다

In [30]:
any_t_perm_contigous

tensor([[[ 0,  3,  6],
         [ 1,  4,  7],
         [ 2,  5,  8]],

        [[ 9, 12, 15],
         [10, 13, 16],
         [11, 14, 17]]])

In [31]:
any_t

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8]],

        [[ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]]])